In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import time
import os
import pandas as pd

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:46.0) "
                  "Gecko/20100101 Firefox/46.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0"
}

In [3]:
def get_link(link):
    link_list = []
    req = requests.Session().get(link, headers=HEADERS).text
    soup = BeautifulSoup(req, "lxml")
    body_page = soup.find('div', class_="col-12 col-lg-9 column-border")
    links = body_page.find_all('figure', class_="story__thumb")
    for i in links:
        link_list.append(i.find('a')['href'])

    return link_list


def get_highlight_link(link):
    link_list = []
    req = requests.Session().get(link, headers=HEADERS).text
    soup = BeautifulSoup(req, "lxml")
    story_highlight1 = soup.find('div', class_="col-12 col-lg-6")
    story_highlight2 = soup.find('div', class_="col-12 col-md-6 col-lg-3")

    links1 = story_highlight1.find_all('figure', class_="story__thumb")
    for i in links1:
        link_list.append(i.find('a')['href'])

    links2 = story_highlight2.find_all('figure', class_="story__thumb")
    for i in links2:
        link_list.append(i.find('a')['href'])

    return link_list


def page_is_empty(link):
    req = requests.Session().get(link, headers=HEADERS).text
    soup = BeautifulSoup(req, 'lxml')
    body_page = soup.find('div', class_="col-12 col-lg-9 column-border")
    articles = body_page.find_all('article', class_="story story--featured story--timeline")
    if len(articles) > 0:
        return False
    else:
        return True


def get_categories(link):
    catergories = []
    req = requests.Session().get(link, headers=HEADERS).text
    soup = BeautifulSoup(req, 'lxml')
    bar1 = soup.find_all('a', class_="link__extend")
    for i in bar1[1:4]:
        catergories.append(i['href'])

    bar2 = soup.find_all('a', class_="nav-link")
    for i in bar2:
        if 'htm' in str(i['href']):
            catergories.append(i['href'])

    return catergories

In [4]:
def crawl(link):
    req = requests.Session().get(link, headers=HEADERS).text
    soup = BeautifulSoup(req, 'lxml')

    try:
        menu = soup.find('h1', class_="category-main").text
    except:
        menu = ''

    try:
        tieude = soup.find('h1', class_="detail__title").text
    except:
        tieude = ''

    try:
        anh = soup.find('figure', class_="detail__avatar").find('img')['src']
    except:
        anh = ''

    try:
        time_web = soup.find('div', class_="detail__meta").text
    except:
        time_web = ''

    try:
        publish_time = datetime.strptime(time_web, '%H:%M %d/%m/%Y').timestamp()
    except:
        publish_time = ''

    try:
        motangan = soup.find('h2', class_="detail__summary").text
    except:
        motangan = ''

    try:
        body = soup.find('div', class_="detail__content").text
    except:
        body = ''


    dictionnary = {
        'Menu': menu,
        'Tieude': tieude,
        'Anh': anh,
        'Lienket': link,
        'Motangan': motangan,
        'Body': body,
        'Publish_time': publish_time,
        'Received_time': datetime.now().timestamp()
    }

    return dictionnary

In [ ]:
vneconomy_categories = get_categories('https://vneconomy.vn/')

for i in vneconomy_categories:
    print(i)
    highlights = get_highlight_link('https://vneconomy.vn/' + i)
    for k in highlights:
        if not ('epaper' in k or 'files' in k):
            try:
                content = crawl('https://vneconomy.vn' + k)
                if float(content['Publish_time']) < float(time.mktime(datetime.now().date().timetuple())):
                    break
                else:
                    json_object = json.dumps(content, indent=4)
                    g = open('/home/khoatm4/Elastic_search/News/News_folder/' + 'vneconomy_' + k[1:] + '.json', 'w+')
                    g.write(json_object)
            except:
                print('https://vneconomy.vn' + k)

    j = 1
    while not page_is_empty():
        links = get_link('https://vneconomy.vn/' + i + '?trang=' + str(j))
        for l in links:
            if not ('epaper' in l or 'files' in l):
                try:
                    content = crawl('https://vneconomy.vn' + l)
                    json_object = json.dumps(content, indent=4)
                    g = open('/home/khoatm4/Elastic_search/News/News_folder/' + 'vneconomy_' + l[1:] + '.json', 'w+')
                    g.write(json_object)

                except:
                    print('https://vneconomy.vn' + l)

        print(j)
        j += 1